In [30]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

path = 'C:/Users/ppang/Documents/dev/TIL/dacon/data/'

train = pd.read_csv(path + 'raw/train.csv')
test = pd.read_csv(path + 'raw/test.csv')

In [2]:
# train
trn_null_col = ['ID','CD','FH2O','FNOX','FOPTIMETHGLY','FOXID','FSO4','FTBN','FUEL','SOOTPERCENTAGE','U100','U75','U50','U25','U20','U14','U6','U4','V100']

trn_str_col = ['COMPONENT_ARBITRARY', 'AG', 'BE']
trn_num_col = ['ANONYMOUS_1','YEAR','SAMPLE_TRANSFER_DAY','ANONYMOUS_2','AL','B','BA','CA','CO','CR','CU','FE','H2O','K','LI','MG','MN','MO','NA','NI','P','PB','PQINDEX','S','SB','SI','SN','TI','V','V40','ZN']
label_col = ['Y_LABEL']
label = train['Y_LABEL']

train = train[trn_str_col+trn_num_col+label_col]
train[trn_str_col] = train[trn_str_col].astype(str)

In [3]:
# test
tst_use_col = ['COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V','V40','ZN']

tst_str_col = ['COMPONENT_ARBITRARY','AG']
# tst_num_col
tst_num_col = ['ANONYMOUS_1','YEAR','ANONYMOUS_2','CO','CR','CU','FE','H2O','MN','MO','NI','PQINDEX','TI','V','V40','ZN']

test = test[tst_use_col]
test[tst_str_col] = test[tst_str_col].astype(str)

## 결측처리

In [4]:
train.fillna(train[['K']].mean(), inplace=True)

### encoding, scaling

In [5]:
mms = MinMaxScaler()

train[trn_num_col] = mms.fit_transform(train[trn_num_col])
trn_str = pd.get_dummies(train[trn_str_col])
train = pd.concat([trn_str,train[trn_num_col]], axis=1)
train['Y_LABEL'] = label


test[tst_num_col] = mms.fit_transform(test[tst_num_col])
tst_str = pd.get_dummies(test[tst_str_col])
test = pd.concat([tst_str,test[tst_num_col]], axis=1)

In [6]:
train.shape

(14095, 49)

In [7]:
test.shape

(6041, 23)

## 저장

In [10]:
train.to_csv(path+'processed/processed_train.csv', index=False)
test.to_csv(path+'processed/processed_test.csv', index=False)

## Datawig

In [11]:
import os
import pandas as pd
import numpy as np
import impyute as impy
from sklearn.preprocessing import LabelEncoder

path = 'C:/Users/ppang/Documents/dev/TIL/dacon/data/'

train = pd.read_csv(path + 'raw/train.csv')
test = pd.read_csv(path + 'raw/test.csv')

In [26]:
lab = LabelEncoder()
temp = train.iloc[:,1:].copy()
temp['COMPONENT_ARBITRARY'] = lab.fit_transform(temp[['COMPONENT_ARBITRARY']])

c:\Users\ppang\anaconda3\envs\pytorch_p39\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [17]:
temp = impy.fast_knn(temp.values)

In [25]:
temp = pd.DataFrame(data=temp, columns=train.columns[1:])
new_train = pd.merge(train.iloc[:,:2], temp.iloc[:,1:], left_index=True, right_index=True)
new_train

,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,...,U25,U20,U14,U6,U4,V,V100,V40,ZN,Y_LABEL
0,TRAIN_00000,COMPONENT3,1486.0,2011.0,7.0,200.0,0.0,3.0,93.0,0.0,...,29.870898,56.063472,239.938149,5488.693107,19415.597262,0.0,13.57623,154.0,75.0,0.0
1,TRAIN_00001,COMPONENT2,1350.0,2021.0,51.0,375.0,0.0,2.0,19.0,0.0,...,2.000000,4.000000,6.000000,216.000000,1454.000000,0.0,13.57623,44.0,652.0,0.0
2,TRAIN_00002,COMPONENT2,2415.0,2015.0,2.0,200.0,0.0,110.0,1.0,1.0,...,0.000000,3.000000,39.000000,11261.000000,41081.000000,0.0,13.57623,72.6,412.0,1.0
3,TRAIN_00003,COMPONENT3,7389.0,2010.0,2.0,200.0,0.0,8.0,3.0,0.0,...,29.870898,56.063472,239.938149,5488.693107,19415.597262,0.0,13.57623,133.3,7.0,0.0
4,TRAIN_00004,COMPONENT3,3954.0,2015.0,4.0,200.0,0.0,1.0,157.0,0.0,...,29.870898,56.063472,239.938149,5488.693107,19415.597262,0.0,13.57623,133.1,128.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,TRAIN_14090,COMPONENT3,1616.0,2014.0,8.0,200.0,0.0,2.0,201.0,1.0,...,29.870898,56.063472,239.938149,5488.693107,19415.597262,0.0,13.57623,135.4,16.0,0.0
14091,TRAIN_14091,COMPONENT1,2784.0,2013.0,2.0,200.0,0.0,3.0,85.0,0.0,...,29.870898,56.063472,239.938149,5488.693107,19415.597262,0.0,14.50000,117.5,1408.0,0.0
14092,TRAIN_14092,COMPONENT3,1788.0,2008.0,9.0,550.0,0.0,6.0,0.0,1.0,...,29.870898,56.063472,239.938149,5488.693107,19415.597262,0.0,13.57623,54.0,1301.0,0.0
14093,TRAIN_14093,COMPONENT2,2498.0,2009.0,19.0,550.0,0.0,2.0,4.0,0.0,...,7.000000,8.000000,100.000000,1625.000000,18890.000000,0.0,13.57623,44.3,652.0,0.0


In [28]:
path

'C:/Users/ppang/Documents/dev/TIL/dacon/data/'

In [29]:
new_train.to_csv(path+'processed/train_impyute.csv', index=False)

In [40]:
new_train.columns

Index(['ID', 'COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR',
       'SAMPLE_TRANSFER_DAY', 'ANONYMOUS_2', 'AG', 'AL', 'B', 'BA', 'BE', 'CA',
       'CD', 'CO', 'CR', 'CU', 'FH2O', 'FNOX', 'FOPTIMETHGLY', 'FOXID', 'FSO4',
       'FTBN', 'FE', 'FUEL', 'H2O', 'K', 'LI', 'MG', 'MN', 'MO', 'NA', 'NI',
       'P', 'PB', 'PQINDEX', 'S', 'SB', 'SI', 'SN', 'SOOTPERCENTAGE', 'TI',
       'U100', 'U75', 'U50', 'U25', 'U20', 'U14', 'U6', 'U4', 'V', 'V100',
       'V40', 'ZN', 'Y_LABEL'],
      dtype='object')

In [41]:
trn_str_col = ['COMPONENT_ARBITRARY']
trn_num_col = ['ANONYMOUS_1', 'YEAR', 'SAMPLE_TRANSFER_DAY', 'ANONYMOUS_2', 'AG', 'AL', 'B', 'BA', 'BE', 'CA', 'CD', 'CO', 'CR', 'CU', 'FH2O', 'FNOX', 'FOPTIMETHGLY', 'FOXID', 'FSO4',
'FTBN', 'FE', 'FUEL', 'H2O', 'K', 'LI', 'MG', 'MN', 'MO', 'NA', 'NI', 'P', 'PB', 'PQINDEX', 'S', 'SB', 'SI', 'SN', 'SOOTPERCENTAGE', 'TI', 'U100', 'U75', 'U50', 'U25', 'U20', 'U14', 'U6', 'U4', 'V', 'V100', 'V40', 'ZN']
label = train['Y_LABEL']

tst_str_col = ['COMPONENT_ARBITRARY']
tst_num_col = ['ANONYMOUS_1','YEAR','ANONYMOUS_2','AG','CO','CR','CU','FE','H2O','MN','MO','NI','PQINDEX','TI','V','V40','ZN']

In [ ]:
mms = MinMaxScaler()

new_train[trn_num_col] = mms.fit_transform(new_train[trn_num_col])
trn_str = pd.get_dummies(new_train[trn_str_col])
new_train = pd.concat([trn_str,new_train[trn_num_col]], axis=1)
new_train['Y_LABEL'] = label


test[tst_num_col] = mms.fit_transform(test[tst_num_col])
tst_str = pd.get_dummies(test[tst_str_col])
test = pd.concat([tst_str,test[tst_num_col]], axis=1)

In [50]:
len(new_train.columns)

56

In [51]:
len(test.columns)

21

In [55]:
new_train.to_csv(path+'processed/processed_train_impyute.csv', index=False)
test.to_csv(path+'processed/processed_test_impyute.csv', index=False)